In [4]:
infer_model = load_model('C:/Users/seanchlin/AI_project/X_ray_cutting_IMT/imt_yolo3.h5')

Instructions for updating:
Colocations handled automatically by placer.


C:\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [43]:
# predict
import os
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
from keras.models import load_model
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pandas as pd
# import logging

input_path   = 'D:/Work/AI_Train_Data/X_Ray_cutting/predict/'
output_path  = 'D:/Work/AI_Train_Data/X_Ray_cutting/output/'
makedirs(output_path)

# Set some parameter
net_h, net_w = 608, 608 # a multiple of 32, the smaller the faster

obj_thresh, nms_thresh = 0.8, 0.45

anchors = [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
labels = ['Mark','X','Y'] #0,1,2

# Load the model
# infer_model = load_model('C:/Users/seanchlin/AI_project/X_ray_cutting_IMT/imt_yolo3.h5')

# Read file
image_paths = []
if os.path.isdir(input_path): 
    for inp_file in os.listdir(input_path):
        image_paths += [input_path + inp_file]
else:
    image_paths += [input_path]

image_paths = [inp_file for inp_file in image_paths if (inp_file[-4:] in ['.jpg', '.png', '.JPG'])]

# the main loop
for image_path in image_paths:
    image = cv2.imread(image_path)
    print(image_path)

# predict the bounding boxes
    boxes = get_yolo_boxes(infer_model, [image], net_h, net_w, anchors, obj_thresh, nms_thresh)[0]

# draw bounding boxes on the image using labels
    draw_boxes(image, boxes, labels, obj_thresh)  

# write the image with bounding boxes to file
#     output_img_path = output_path + image_path.split('/')[-1]
    output_img_path = output_path +'predict_'+image_path.split('/')[-1]
    cv2.imwrite(output_img_path, np.uint8(image))
    img = cv2.imread(output_img_path)[:,:,::-1]
    print(output_img_path)
    

D:/Work/AI_Train_Data/X_Ray_cutting/predict/t001_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t001_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t001_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t001_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t002_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t002_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t002_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t002_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t003_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t003_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t003_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t003_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t004_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t004_03.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t004_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/output/predict_t004_04.jpg
D:/Work/AI_Train_Data/X_Ray_cutting/predict/t005

In [42]:
labs=[]
for m in range(len(boxes)):
    if boxes[m].score !=-1:
        print(boxes[m].label,boxes[m].xmin,boxes[m].ymin,boxes[m].xmax,boxes[m].ymax)

1 -3 188 509 194
2 234 29 251 291
0 244 158 327 192
1 -25 185 329 197
2 237 113 251 432
